In [1]:
from Project_code import *

board = chess.Board()

pre_mate = chess.Board('rnbqkbnr/pppp1ppp/8/4p3/6P1/5P2/PPPPP2P/RNBQKBNR b KQkq - 0 2')

version()

V 0.2.10


In [ ]:
games = loadFromRepository()
library = gamesToLibrary(games)

In [3]:
training_indexes = list(range(1,len(library[0])))
np.random.shuffle(training_indexes)
train_data = [library[0][training_indexes],library[1][training_indexes]]
train_pos =[]
for n,i in enumerate(train_data[0]):
    train_pos += [position_bitboard.getValues(i),]
train_lab = train_data[1]

TypeError: list indices must be integers or slices, not list

In [ ]:
train_pos = np.array(train_pos, dtype=np.float32)
train_lab = np.array(train_lab, dtype=np.float32)

In [ ]:
train_pos= train_pos[0:100]
train_lab=train_lab[0:100]

In [2]:
first_model = Model([
    tf.keras.layers.Flatten(input_shape=(13,8,8),name="flatten"),
    tf.keras.layers.Dense(100,activation="relu",name="layer2"),
    tf.keras.layers.Dense(1,activation="tanh",name="output")
])

first_model.setName("first_model")

first_model.compile(optimizer='adam',
              loss="mean_squared_error",
              metrics=['accuracy'])


In [ ]:
first_model

In [94]:
first_model.save()

In [6]:
first_model.fit(x = train_pos,
          y = train_lab)

NameError: name 'train_pos' is not defined

In [10]:
model.predict(np.array([position_bitboard.getValues(board)]))

1/1 [==============================] - 0s 168ms/step


array([[0.32714742]], dtype=float32)

In [50]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 768)               590592    
                                                                 
 layer2 (Dense)              (None, 100)               76900     
                                                                 
 output (Dense)              (None, 1)                 101       
                                                                 
Total params: 667593 (2.55 MB)
Trainable params: 667593 (2.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [2]:
time.time()

1718674123.1001225

In [55]:
def ID_alphabeta_branch(board,n,eval,alpha=-2,beta=2): # iterative deepening alpha-beta prunning        
    def id_search(board,n,eval,alpha,beta):
        return classified_move_list #[([move1.1,move1.2],value1),([move2.1,move2.2],value2)]
    
    
    poss_moves = list(board.legal_moves)
    evals = []
    if len(poss_moves)>0: #Has possible moves -> evals each Node/Leaf    
        if n==1: #Leafs          
            if board.turn:
                value = -5
                for move in poss_moves:
                    t_board = chess.Board(board.fen())
                    t_board.push(move)
                    t_value = eval(t_board)
                    if t_value > value:
                        value = t_value
                        best_move = move
                    elif t_value == value:
                        r = random.random()
                        if r > 0.5:
                            value = t_value
                            best_move = move
            else:
                value = 5
                for move in poss_moves:
                    t_board = chess.Board(board.fen())
                    t_board.push(move)
                    t_value = eval(t_board)
                    if t_value < value:
                        value = t_value
                        best_move = move
                    elif t_value == value:
                        r = random.random()
                        if r > 0.5:
                            value = t_value
                            best_move = move
                                          
        else: #Branch Nodes
            if board.turn:
                value = -5
                for move in poss_moves:
                    t_board = chess.Board(board.fen())
                    t_board.push(move)
                    t_value = ab_branch(t_board,n-1,eval,alpha,beta)[1]
                    if t_value > value:
                        value = t_value
                        best_move = move
                    elif t_value == value:
                        r = random.random()
                        if r > 0.1:
                            value = t_value
                            best_move = move
                    if value > beta:
                        break
                    alpha = max(alpha,value)
                    
                         
            else:
                value = 5
                for move in poss_moves:
                    t_board = chess.Board(board.fen())
                    t_board.push(move)
                    t_value = ab_branch(t_board,n-1,eval,alpha,beta)[1]
                    if t_value < value:
                        value = t_value
                        best_move = move
                    elif t_value == value:
                        r = random.random()
                        if r > 0.1:
                            value = t_value
                            best_move = move
                    if value < alpha:
                        break
                    beta = min(beta,value) 
                    
                    
    else:
        if board.is_game_over():
            if board.is_checkmate():
                value = 2*(board.turn*(-2)+1)
                best_move = "checkmate?"
            elif board.is_insufficient_material():
                value = 0
                best_move = "insf material?"
            elif board.is_stalemate():
                value = 0
                best_move = "stalemate?"
        else:
            print(board.fen())
            
            
    return best_move,value

In [3]:
def ID_AB_prunning(board,n,Evaluator,alpha=-2,beta=2): #alpha-beta prunning        
    poss_moves = list(board.legal_moves)
    if len(poss_moves)>0: #Has possible moves -> evals each Node/Leaf 
        
        list_boards = []
        
        for move in poss_moves:
            t_board = chess.Board(board.fen())
            t_board.push(move)
            list_boards += [t_board,]
            values = Evaluator.evaluate(list_boards)
        
        sorted_indices = sorted(range(len(values)),key=lambda k: values[k])
        values = [values[i] for i in sorted_indices]
        poss_moves = [poss_moves[i] for i in sorted_indices]
        
        
                                         
        if n>1: #Branch Nodes
            if board.turn: #White
                value = -5
                for move in poss_moves:
                    t_board = chess.Board(board.fen())
                    t_board.push(move)
                    t_value = AB_prunning(t_board,n-1,Evaluator,alpha,beta)[1]
                    if t_value > value:
                        value = t_value
                        best_move = move
                    elif t_value == value:
                        r = random.random()
                        if r > 0.1:
                            value = t_value
                            best_move = move
                    if value > beta:
                        break
                    alpha = max(alpha,value)        
                         
            else: #Black
                value = 5
                for move in poss_moves:
                    t_board = chess.Board(board.fen())
                    t_board.push(move)
                    t_value = AB_prunning(t_board,n-1,Evaluator,alpha,beta)[1]
                    if t_value < value:
                        value = t_value
                        best_move = move
                    elif t_value == value: #better move
                        r = random.random()
                        if r > 0.1:
                            value = t_value
                            best_move = move
                    if value < alpha: #equally good move
                        break
                    beta = min(beta,value) 
                    
        elif n==1: #Leafs                        
            if board.turn: #White
                value = max(values)
                moves = [i for i, j in enumerate(values) if j == value]
            else: #black
                value = min(values)
                moves = [i for i, j in enumerate(values) if j == value]         
            best_move = poss_moves[random.choice(moves)]
        
        
    else:
        if board.is_game_over():
            if board.is_checkmate():
                value = 2*(board.turn*(-2)+1)
                best_move = "checkmate?"
            elif board.is_insufficient_material():
                value = 0
                best_move = "insf material?"
            elif board.is_stalemate():
                value = 0
                best_move = "stalemate?"
        else:
            print(board.fen()) 
    return best_move,value


In [6]:
n = 0.1
import numpy as np

In [69]:
d = {chess.PAWN:1,chess.KNIGHT:3,chess.BISHOP:3,chess.ROOK:5,chess.QUEEN:9,chess.KING:0.1}
weights = []
for move in board.legal_moves:
    weights += [d[board.piece_type_at(move.from_square)],]
weights = np.array(weights)/sum(weights)#
weights

array([0.10714286, 0.10714286, 0.10714286, 0.10714286, 0.03571429,
       0.03571429, 0.03571429, 0.03571429, 0.03571429, 0.03571429,
       0.03571429, 0.03571429, 0.03571429, 0.03571429, 0.03571429,
       0.03571429, 0.03571429, 0.03571429, 0.03571429, 0.03571429])

In [3]:
def weighted_MCTS(board,depth,Evaluator,param,slave=False): #param needs to be adressed later
    if type(depth)!=int:
        raise TypeError("depth needs to be integer equal or higher than 1 ")
    #if param>1 or param<=0:
    #    raise ValueError("MCTS parameter needs to be belong in ]0,1]")
    poss_moves = list(board.legal_moves)
    
    
    
    if len(poss_moves)>0:
        if slave:
            d = {chess.PAWN:1,chess.KNIGHT:3,chess.BISHOP:3,chess.ROOK:5,chess.QUEEN:9,chess.KING:0.1}
            weights = []
            for move in board.legal_moves:
                weights += [d[board.piece_type_at(move.from_square)],]
            weights = np.array(weights)/sum(weights)
            
            n_moves = math.ceil(len(poss_moves)*param)
            chosen_moves = np.random.choice(poss_moves,n_moves,replace=False,p=weights)# replace can be changed in a different algorithm
        else:
            chosen_moves=poss_moves
        list_boards = []
        move_board_list = []
        best_move = 0 #if it's relevant it will be changed later in the function
        if depth>1: #Branch nodes
            
            for move in chosen_moves:#search every node
                t_board = chess.Board(board.fen())
                t_board.push(move)
                list_boards = list_boards+weighted_MCTS(t_board,depth-1,Evaluator,param,True)[1]# get the list of "all" boards
                if not slave:
                    move_board_list += [move,]*len(list_boards)#matching boards to moves
            
                
        elif depth==1: #Leafs  
            list_boards = []
            for move in chosen_moves:
                t_board = chess.Board(board.fen())
                t_board.push(move)
                list_boards += [t_board,]
            if not slave:
                move_board_list += [move,]*len(list_boards)
                 
    else: #No move left
        if board.is_game_over():
            list_boards = [board]#return itself 
        else:
            raise ValueError("board has no moves but is not over:",board.fen())
                
    
    if not slave:
        values = Evaluator.evaluate(list_boards)
        if board.turn: #White
            value = np.max(values)
            moves = np.where(values == value)[0]
        else: #black
            value = np.min(values)
            moves = np.where(values == value)[0]
        best_move = move_board_list[random.choice(moves)]
    
    return best_move,list_boards
    

In [3]:
simulateChessGame(basic_bot,basic_bot)

(0, Board('1n1k3r/1P6/8/5n2/8/8/BK6/R5R1 b - - 20 50'))

In [4]:
class SearcherDirector:
    @staticmethod
    def minimax(depth):
        descr = None
        return Searcher("Minimax, depth "+str(depth),descr,minimax,depth,None)
    
    @staticmethod
    def AB_prunning(depth):
        descr = None
        return Searcher("Alpha-Beta prunning, depth "+str(depth),descr,AB_prunning,depth,None)
    
    @staticmethod
    def ID_AB_prunning(depth):
        descr = None
        return Searcher("Iterative deepening Alpha-Beta prunning, depth "+str(depth),descr,ID_AB_prunning,depth,None)
    
    @staticmethod
    def MCTS(depth,param):
        descr = None
        return Searcher("Monte Carlo Tree Search, depth "+str(depth)+"  factor "+str(param),descr,basic_MCTS,depth,param)
    
    @staticmethod
    def weighted_MCTS(depth,param):
        descr = None
        return Searcher("Weighted Monte Carlo Tree Search, depth "+str(depth)+"  factor "+str(param),descr,weighted_MCTS,depth,param)

In [5]:
n=5
mcts_searcher = SearcherDirector.weighted_MCTS(n,0.5)

nn_eval = NNEvaluator("","",[TraitDirector.PositionBitboard()],first_model)
mcts_bot = BotPlayer(nn_eval,mcts_searcher)

In [6]:
mcts_searcher.param

0.5

In [7]:
mcts_bot.doMove(board)

TypeError: unsupported operand type(s) for *: 'int' and 'NNEvaluator'

In [39]:
n=4
ab_searcher_n = Searcher(AB_prunning,n)
basic_n_bot = BotPlayer(nn_eval,ab_searcher_n)

n_ab_searcher_n = Searcher(n_AB_prunning,n)
n_basic_n_bot = BotPlayer(nn_eval,ab_searcher_n)
id_searcher_n = Searcher(ID_AB_prunning,n)
id_n_bot = BotPlayer(nn_eval,id_searcher_n)


nn_searcher_n = Searcher(nn_minimax,n)

nn_n_bot = BotPlayer(nn_eval,nn_searcher_n)

In [22]:
basic_n_bot.doMove(board)

Move.from_uci('e2e3')

In [38]:
n_basic_n_bot.doMove(board)

Move.from_uci('e2e3')

In [40]:
nn_n_bot.doMove(board)

KeyboardInterrupt: 

In [9]:
id_n_bot.doMove(board)

KeyboardInterrupt: 